<a href="https://colab.research.google.com/github/world970511/RIDIBOOKS_romance_webnovel_review_Sentiment_Analysis/blob/main/anlay_ridi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import re
import urllib.request
from sklearn.model_selection import train_test_split


urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/RIDIBOOKS_romance_webnovel_review_Sentiment_Analysis/main/data/result_all_data.txt",
                        filename="result_all_data.txt")
all_data = pd.read_csv('result_all_data.txt',encoding='utf-8')
print(all_data)

print('\n===================\n')
print('초반 데이터 확인 :',len(all_data))
all_data.drop_duplicates(subset=['review'], inplace=True)#중복된 리뷰들을 제거한다
print('중복 제거 확인 :',len(all_data))

all_data = all_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print('null값이 존재하는가?=',all_data.isnull().values.any()) # Null 값이 존재하는지 확인
print('\n===================\n')

train_data, test_data = train_test_split(all_data, test_size = 0.25, random_state = 42)#3:1로 데이터 분할
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

print('\n===================\n')
train_data.groupby('rate').size().reset_index(name = 'count')
test_data.groupby('rate').size().reset_index(name = 'count')

train_data=train_data.fillna(" ")#훈련 데이터에서 한글과 공백을 제외하고 제거
train_data['review']=train_data['review'].apply(lambda x:re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))
train_data['review']=train_data['review'].apply(lambda x:re.sub('^ +',"",x))
train_data['review'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how='any')
print('전처리 후 훈련용 샘플의 개수 :',len(train_data))

test_data=test_data.fillna(" ")#테스트 데이터에서 한글과 공백을 제외하고 제거
test_data['review']=test_data['review'].apply(lambda x:re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))
test_data['review']=test_data['review'].apply(lambda x:re.sub('^ +',"",x))
test_data['review'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))
print('\n===================\n')

test_data.to_csv('test.txt',mode='w',index=False)
train_data.to_csv('train.txt',mode='w',index=False)




        rate                                             review
0          1  믿고 보는 작가님 짱  남주 여주 넘 좋아요  여주때문에 안달 못하는 모습도 재미있...
1          1                                        기대됩니다 잘읽을께용
2          0                                하아ㅜㅜ 진짜 남주가 로봇같아요ㅜㅜ
3          0   필력이 진짜... 없어요 이런 판타지 로맨스를 좋아해서 기대하면서 봤는데 휴......
4          0         갈수록 지루하고 유치해져요.  길지도 않은 책인데 끝까지 못읽고 포기했어요.
...      ...                                                ...
110863     1            재밋게 잘봣어요.ㅎ너무빨리읽어져서아쉽네요 작가님다른작품도 찾아봐야겟어요
110864     0  .개취라지만 납득안감..여주는 한장넘어갈때마다 울고 눈물흘리고..혼자 걷지도 못하는...
110865     1                              좋아하는 작가님 신작입니다! 기대돼요~
110866     1                                작가님 책 다 봤어요. 잼있어요~~
110867     0  ㅋㅋㅋ제목 잘못된수준이에요ㅋㅋ 꽃잎은 젖어든다가 아니라 젖어있다라던가 마를날이 없다...

[110868 rows x 2 columns]


초반 데이터 확인 : 110868
중복 제거 확인 : 110868
null값이 존재하는가?= False


훈련용 리뷰의 개수 : 83151
테스트용 리뷰의 개수 : 27717


전처리 후 훈련용 샘플의 개수 : 83100
전처리 후 테스트용 샘플의 개수 : 27697




In [5]:
pip install konlpy

     |████████████████████████████████| 19.4MB 7.9MB/s 
     |████████████████████████████████| 460kB 38.5MB/s 
     |████████████████████████████████| 92kB 13.7MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [26]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))


train_data=pd.read_csv('train.txt',encoding='utf-8')
test_data=pd.read_csv('test.txt',encoding='utf-8')

okt = Okt()
stopwords = ['의','가','이','은','들','는','좀','원','권','과','도','를','으로','자','에','와','한','하다','뽕빨','제','금']

X_train = []
for sentence in train_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

X_test = []
for sentence in test_data['review']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

print('\n===================\n')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

y_train = np.array(train_data['rate'])
y_test = np.array(test_data['rate'])

drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))

print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))

max_len = 85
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)



단어 집합(vocabulary)의 크기 : 32208
등장 빈도가 2번 이하인 희귀 단어의 수: 17419
단어 집합에서 희귀 단어의 비율: 54.08283656234476
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.25971812003049
단어 집합의 크기 : 14790


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


83072
리뷰의 최대 길이 : 725
리뷰의 평균 길이 : 20.177881837442218
전체 샘플 중 길이가 85 이하인 샘플의 비율: 97.3432684899846


In [27]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Epoch 1/15
1108/1108 [==============================] - 18s 15ms/step - loss: 0.2947 - acc: 0.8815 - val_loss: 0.2642 - val_acc: 0.8955

Epoch 00001: val_acc improved from -inf to 0.89552, saving model to best_model.h5
Epoch 2/15
1108/1108 [==============================] - 15s 14ms/step - loss: 0.2313 - acc: 0.9109 - val_loss: 0.2428 - val_acc: 0.9056

Epoch 00002: val_acc improved from 0.89552 to 0.90563, saving model to best_model.h5
Epoch 3/15
1108/1108 [==============================] - 14s 13ms/step - loss: 0.2093 - acc: 0.9204 - val_loss: 0.2343 - val_acc: 0.9080

Epoch 00003: val_acc improved from 0.90563 to 0.90803, saving model to best_model.h5
Epoch 4/15
1108/1108 [==============================] - 14s 13ms/step - loss: 0.1932 - acc: 0.9282 - val_loss: 0.2339 - val_acc: 0.9112

Epoch 00004: val_acc improved from 0.90803 to 0.91122, saving model to best_model.h5
Epoch 5/15
1108/1108 [==============================] - 14s 13ms/step - loss: 0.1806 - acc: 0.9345 - val_loss: 0.22

In [28]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

  
if __name__ == "__main__": 
  while(True):
    new_sentence=input("리뷰 테스트(종료하고 싶다면 n을 쳐주세요):")
    if  new_sentence !='n':
      try:
        sentiment_predict(new_sentence) 
      except:
        break
        print('stop')
    else: break

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):작가님 외전 주세요 제빌
96.98% 확률로 긍정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):가독성이 너무 떨어져요
99.11% 확률로 부정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):재밌게 읽었습니다
97.95% 확률로 긍정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):설정도 좋고 필력도 좋으신데 뒤로 가면 갈수록 이야기가 이상해지네요
84.61% 확률로 부정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):스토리라인이 이상해요;; 돈아깝네요
99.66% 확률로 부정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):여주고 남주고 둘다 이상한거 같은데
96.08% 확률로 부정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):재밌어요. 대여해놓고  왠지 미루다미루다 봤는데..왠걸 재밌어요. 근데 댓글들처럼 복수가아숩네요.
97.96% 확률로 긍정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):기대치가 너무 컸나봐요
80.20% 확률로 부정 리뷰입니다.

리뷰 테스트(종료하고 싶다면 n을 쳐주세요):n
